# Running XFOIL with Python

In conducting a study of the aerodynamic performance of an indoor model, we need good data on the flight characteristics of the chosen airfoils. In this section we will present Python code that will use Mark Drela's *XFOIL*{cite}`drela1989` program to produce the needed data for airfoil sections we might use on our models.

*XFOIL* is a *Fortran* program that can be compiled on any platform that has the **fortran** compiler installed. I use the *Free Software Foundation* **gfortran** compiler on my systems. Getting this program ready to run on your platform will be covered elsewhere. For this section, I will assume that **xfoil** is available on your system and it can be run from the *command line*.

## Launching a Python Subprocess

**XFOIL** is a command-line interactive program written in *Fortran*. There are several ways to get *Python* to run such external programs and send commands to it and retrieve the normal console output for processing in your program.

I tried a few of these methods, and finally decided to use a scheme I found in the **pyxfoil** program created by the *MIT Hoburg Research Group*{cite}`hoburg2015`. **pyxfoil** manages *XFOIL* using the **pexpect** package.

The *Python* **pexpect** package is designed to run interactive external programs. Unfortunately, running **pexpect** in of a *Jupyter* notebook page it not possible, so I created a *Python* class to manage running *XFOIL*. The actual code created for this project is in the project *GitHub* repository.

In what follows, I will demonstrate using this code.

In [1]:
import sys
sys.path.insert(0,'../../')
from mmtime.XfoilRunner import XfoilRunner              

We create a running *XFOIL* process by passing in the desired airfoil, Peynolds Number, and an iteration count used to calculate boundary layer effects.

In [2]:
xf = XfoilRunner('flat0001', 3000, 200)

The **xf** variable is now controlling a running **xfoil** process. The next step loads airfoil data into *XFOIL*.

In [3]:
xf.set_airfoil()

WE are not ready to generate the airfoil data. The code is going to try to generate airfoil performance data for angle of attacks ranging from -20 degrees to 20 degrees. (I will modify this later to let the user decide on these values.)

In [4]:
xf.gen_polar()

Generating polar for flat0001


The polar data is stored in a text file in the airfoil data directory. It is stamped with the Reynolds NUmber so we can generate multiple polars as needed for the study.

We can now terminate the background process:

In [5]:
xf.quit()

Let's take a peek at the polar data file generated:

In [6]:
!cat ../../mmtime/data/airfoils/flat0001/flat0001_RE3000.pol

  
       XFOIL         Version 6.99
  
 Calculated polar for: flat0001                                        
  
 1 1 Reynolds number fixed          Mach number fixed         
  
 xtrf =   1.000 (top)        1.000 (bottom)  
 Mach =   0.000     Re =     0.003 e 6     Ncrit =   9.000  9.000
  
   alpha    CL        CD       CDp       CM     Top_Xtr  Bot_Xtr  Top_Itr  Bot_Itr
  ------ -------- --------- --------- -------- -------- -------- -------- --------
 -20.000  -0.9648   0.25215   0.34780   0.1274   1.0000   1.0000   1.0000 263.0000
 -18.000  -0.8888   0.22769   0.28615   0.1104   1.0000   1.0000   1.0000 263.0000
 -16.000  -0.8112   0.20346   0.23072   0.0941   1.0000   1.0000   1.0000 263.0000
 -14.000  -0.7323   0.17935   0.18144   0.0788   1.0000   1.0000   1.0000 263.0000
 -12.000  -0.6533   0.15524   0.13853   0.0645   1.0000   1.0000   1.0000 263.0000
 -10.000  -0.5764   0.13110   0.10198   0.0512   1.0000   1.0000   1.0000 263.0000
  -8.000  -0.5040   0.10703   0.07153   

The **XfoilRunner** code generates a time stamped log file that is stored in a **logs** subdirectory with the airfoil data. This file can be very useful when diagnosing problems getting **xfoil** to process an airfoil.

```{warning}
The log file can be very long, especially if *XFOIL* has issues generating a particular setting.
```